## Data Cleaning With Pandas

Sources of Missing Values

Standard Missing Values

Summarizing Missing Values

Analyzing Obesity in England

Time Series

Basic Time Series Manipulation

Time Series on Dataset


Sumber dari data yang hilang : 

    Pengguna lupa mengisi kolom.
    Data hilang saat mentransfer secara manual dari database lama.
    Terjadi kesalahan pemrograman.
    Pengguna memilih untuk tidak mengisi bidang yang terkait dengan keyakinan mereka 
    tentang bagaimana hasil akan digunakan atau ditafsirkan.

kalian mulai membersihkan kumpulan data, ada baiknya kalian memahami data secara umum. Setelah itu, kalian bisa menyusun rencana untuk membersihkan data.

Mulailah dengan menanyakan pertanyaan-pertanyaan berikut:

     Apa saja fiturnya?
     Apa tipe yang diharapkan (int, float, string, boolean)?
     Apakah ada data yang hilang (nilai yang dapat dideteksi Panda)?
     Apakah ada jenis data lain yang hilang yang tidak begitu jelas (tidak dapat dengan mudah dideteksi dengan Pandas)?

In [ ]:
pip install requests

In [ ]:
import requests

def downloadDataSet(url,destNameFile): 
    """
    Download Data Set
    """
    response = requests.get(url)
    if response.status_code == 200: 
        with open(destNameFile, 'wb') as f:
            f.write(response.content)
        
        response.close()
        return True 
    
    response.close()
    print("Request Error , http statuscode : ",response.status_code)
    return False 

In [ ]:
url = "https://raw.githubusercontent.com/ardhiraka/PFDS_sources/master/property_data.csv"


In [ ]:
namaFile = "property_data.csv"
status = downloadDataSet(url, namaFile)
print(status)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(namaFile)
df.head(10)

fitur-fitur pada kolom adalah:
* ST_NUM: Street number
* ST_NAME: Street name
* OWN_OCCUPIED: Pemiliknya menempati ? 
* NUM_BEDROOMS: Number of bedrooms
* NUM_BATH : number bathrooms 

SQ_FT : square foot 

Tipe yang diharapkan adalah sebagai berikut

In [ ]:
df.info()

* ST_NUM: float or int… some sort of numeric type
* ST_NAME: string
* OWN_OCCUPIED: string… Y (“Yes”) or N (“No”)
* NUM_BEDROOMS: float or int, a numeric type

### Standard Missing Values

Standard missing values adalah missing values yang dapat di deteksi Pandas

### penggunaan apply dan loc

In [ ]:
dfa = pd.DataFrame([[2,3,1],
                    [3,2,2],
                    [2,4,4]], columns=list('ABC'))
dfa

In [ ]:
dfa.apply(lambda x : x['A'], axis=1)

In [ ]:
dfa.apply(lambda x : x['A'], axis = 0) # Error KeyError: 'A'

In [ ]:
dfa.apply(lambda x : x, axis = 1)

In [ ]:
dfa.iloc[0:1,]

In [ ]:
#Access a group of rows and columns by label(s) or a boolean array 
dfa.loc[0]   # default axis = 0 .ndim  1 dim ,   shape 3,1

In [ ]:
dfa.loc[0:1] # 2 dimensi 3, . m:n n is include

In [ ]:
dfa.iloc[0]

In [ ]:
# kita akan memberikan background warna merah pada data frame varibale dfa
# jika data tersebut harus lebih besar dari baris 0 [2, 3, 1]
# try change axis 
# 2 >  [2,3,1]
# 3 > [2,3,1]
# 1 > 
# axis = 0 kolom 
# axis = 1 baris
dfa.style.apply(lambda x: ["background: red" if v >= x.iloc[0] else "" for v in x], axis = 0)

In [ ]:
dfx = pd.DataFrame([[4,9]] * 3, columns=['A', 'B'])
dfx

In [ ]:
dfx.apply(np.sqrt)

In [ ]:
dfx.apply(np.sum, axis=0)  # apply untuk tipa kolom

In [ ]:
dfx.apply(np.sum, axis=1)  # apply untuk tiap baris


In [ ]:
# Returning a list-like will result in a Series [1.2]
dfx.apply(lambda x: [1, 2], axis=1)

In [ ]:
#Passing ``result_type='expand'`` will expand list-like results to columns of a Dataframe
dfx.apply(lambda x: [1, 2], axis=1, result_type='expand')

In [ ]:
dfx.apply(lambda x: pd.Series([1, 2], index=['foo', 'bar']), axis=1)

In [ ]:
df.head(10)

In [ ]:
street_num = df.loc[:,['ST_NUM']]
street_num

In [ ]:
import math 
def highlight_column_nan(column):    
    """
    Function untuk me return background-color
    """
    highlight = 'background-color: palegreen;'
    default = '' 
    #return [highlight if str(v) == str(1e400*0)  else default for v in column]
    return [highlight if math.isnan(v)  else default for v in column]
    
#df.style.apply(highlight_column_nan, subset=['ST_NUM'], axis=0)

In [ ]:
# menampilkan beberapa kolom
street_num = df.loc[:,['ST_NUM','ST_NAME','NUM_BEDROOMS','OWN_OCCUPIED']]
street_num

df.apply?
- 0 or 'index': apply function to each column
- 1 or 'columns': apply function to each row
 sedikit berbeda dengan dataframe axis = 0 baris, axis = 1 kolom

In [ ]:
# melakukan apply function highlight_column_nan
street_num.style.apply(highlight_column_nan, subset=['ST_NUM'], axis=0)

In [ ]:
# menampilkan st_num
df['ST_NUM']

In [ ]:
# melakukan pengecekan is null/isna
df['ST_NUM'].isnull()

In [ ]:
df['ST_NUM'].isna()

### Non-Standard Missing Values
kita coba melihat ada beberapa data yang tidak standar yang akan menjadikan missing values pada num_bedrooms

In [ ]:
import math 

missing_values = ["n/a", "na", "--"]

#str(1e400*0) == NaN
def highlight_column_missing_values(column):    
    highlight = 'background-color: yellow;'
    default = '' 
    #return [highlight if str(v) == str(1e400*0)  else default for v in column]
    return [highlight if str(v) == str(1e400*0) or v in missing_values else default for v in column]
    
#df.style.apply(highlight_column_nan, subset=['ST_NUM'], axis=0)

In [ ]:
street_num.style.apply(highlight_column_missing_values, subset=['NUM_BEDROOMS'], axis=0)

Di kolom ini ada empaat missing values:
- n/a
- NA
- __
- na dari bagian sebelumnya, kita tahu bahwa pandas akan mengenali "NA" sebagai missing value, tapi bagaimana dengan yang lain? Mari lihat, dengan pengecekan is null

In [ ]:
df['NUM_BEDROOMS']

In [ ]:
# kita coba lakukan pengecekan menggunakan is null
df['NUM_BEDROOMS'].isnull()
# perhatikan baris 7 dan 8

Pada index 7 dan 8 is null tidak true dengan kata lian tidak di identifikasikan sebagai NA

In [ ]:
# membuat list missing values
missing_values = ["n/a", "na", "--"]

In [ ]:
df = pd.read_csv(namaFile, na_values= missing_values) # di definisakn debgai kategory NA

In [ ]:
df['NUM_BEDROOMS']

In [ ]:
df['NUM_BEDROOMS'].isnull()

Kali ini, semua format berbeda dikenai sebagai missing values

### Unexpected Missing Values

Sejauh ini kita telah melihat missing values, dan non-standard missing values. Bagaimana jika kita memiliki tipe yang tidak terduga? Misalnya, jika fitur kita diharapkan berupa string, tetapi malah jenisnya numerik, maka secara teknis ini juga merupakan missing value.

Mari kita lihat kolom "Owner Occupied" untuk mengetahuinya.

In [ ]:
values_real = ["Y", "N", "y", "n"]

def highlight_column_creteria_values(column):
    highlight = 'background-color: yellow';
    default = ''
    #return [highlight if str(v) == str(1e400*0)  else default for v in column]
    return [default if v in values_real else highlight for v in column]

In [ ]:
street_num.style.apply(highlight_column_creteria_values, subset=['OWN_OCCUPIED'], axis=0)

In [ ]:
df['OWN_OCCUPIED']

In [ ]:
df['OWN_OCCUPIED'].isnull()

Di baris keempat, ada angka 12. Respons untuk Owner Occupied jelas harus berupa string (Y atau N), jadi tipe numerik ini berupa missing value.

Contoh ini sedikit lebih rumit sehingga kita perlu memikirkan strategi untuk mendeteksi jenis nilai yang hilang ini. Ada sejumlah pendekatan berbeda, tetapi inilah cara kita akan mengatasinya.

* Loop pada kolom OWN_OCCUPIED
* Kita coba ubah semua entry menjadi integer
* Jika entry tidak dapat diubah menjadi integer, tandai sebagai missing value
* Jika tidak dapat menjadi integer, maka kita tau kalau entry adalah string, keep going

Mari kita lihat kodenya dan kemudian kita akan membahasnya secara mendetail.

In [ ]:
cnt=0
for row in df['OWN_OCCUPIED']:
    try:
        int(row)
        df.loc[cnt, 'OWN_OCCUPIED']=np.nan
    except ValueError:
        pass
    cnt+=1

In [ ]:
df.head(9)

Dalam kode diatas, kita looping setiap entri di kolom "Owner Occupied". Untuk mencoba dan mengubah entri menjadi integer, kita menggunakan int(row).

Jika nilai dapat diubah menjadi bilangan bulat, kita mengubah entri menjadi missing value menggunakan np.nan Numpy.

Di sisi lain, jika tidak dapat diubah menjadi bilangan bulat, kita pass dan keep going.

Kalian akan melihat bahwa kita menggunakan try dan except ValueError. Ini disebut exception handling, dan kita menggunakan ini untuk menangani errors.

Jika kita mencoba dan mengubah entri menjadi integer dan tidak dapat diubah, ValueError akan di return, dan kode akan berhenti. Untuk mengatasi ini, kita menggunakan exception handling untuk mengenali error ini, dan terus berjalan.


### Summarizing Missing Values

Setelah kita membersihkan missing values, kita mungkin ingin melihat summary-nya. Misalnya, kita mungkin ingin melihat jumlah total missing values untuk setiap feature.

In [ ]:
df.isnull().sum()

In [ ]:
# Mendapatkan jumlah missing value
df.isnull().sum().sum()

#### Replacing

Seringkali kalian harus memikirkan bagaimana kalian menangani missing values.

Terkadang kalian hanya ingin menghapus baris tersebut, di lain waktu kalian menggantinya.

In [ ]:
df

In [ ]:
# maybe you just want to fill in missing values with a single value
df['ST_NUM'].fillna(125, inplace=True)

Kemungkinan besar, kita mungkin ingin melakukan imputasi berbasis lokasi/location based imputation. Inilah cara kita melakukannya.

In [ ]:
df.loc[2, 'ST_NUM'] = 125

Cara yang sangat umum untuk mengganti missing values menggunakan median

In [ ]:
median = df['NUM_BEDROOMS'].median()
median

In [ ]:
df['NUM_BEDROOMS'].fillna(median, inplace=True)

In [ ]:
df.head(9)

### Data Cleaning Data set tumpahan oli



https://github.com/jbrownlee/Datasets/blob/master/oil-spill.csv



#### Latihan number of unique

In [ ]:
import pandas as pd

# creating the first dataframe
df = pd.DataFrame({'A':[14,4,5,4,1],
                   'B':[5,2,54,3,2],
                   'C':[20,20,7,3,8],
                   'D':[14,3,6,2,6]})
df

In [ ]:
# gunakan fungsi dataframe.nunique() untuk menemukan nilai unik di sepanjang sumbu kolom.
df.nunique(axis=1)

In [ ]:
df.nunique(axis=0)

In [ ]:
df = pd.DataFrame({'A':[1,2,3],
                   'B':[1,1,1]})
df
df.nunique()

### Identifikasi kolom yang mengandung Nilai Tunggal
Kolom yang memiliki satu pengamatan atau nilai mungkin tidak berguna untuk pemodelan. 

Kolom atau prediktor ini disebut prediktor varian nol seolah-olah kita mengukur varians (nilai rata-rata dari mean), itu akan menjadi nol.

Ketika sebuah prediktor berisi nilai tunggal, kami menyebutnya prediktor varian nol karena benar-benar tidak ada variasi yang ditampilkan oleh prediktor

In [ ]:
# summarize the number of unique values for each column using numpy
import numpy as np
from numpy import loadtxt
from numpy import unique
# load the dataset
data = loadtxt('https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv', delimiter=',')
# summarize the number of unique values in each column 

print(type(data))

for i in range(data.shape[1]):
    print(i, len(unique(data[:, i])))

In [ ]:
from pandas import read_csv
# load the dataset
df = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv', header=None)
# summarize the number of unique values in each column 
print(df.nunique())

In [ ]:
df.head()

In [ ]:
#get number of unique values for each column
jmlunik = df.nunique()
print(jmlunik)

In [ ]:
# record columns to delete
# for i,v in enumerate(counts) looping dulu i key , v value 
#      if v == 1  
#          i 
to_del = [i for i,v in enumerate(counts) if v == 1]
print(to_del)

### Delete Columns That Conatain a Single Value

In [ ]:
from pandas import read_csv
# load the dataset
df = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv', header=None)
print(df.shape)
# get number of unique values for each column 
counts = df.nunique()
# record columns to delete
to_del = [i for i,v in enumerate(counts) if v == 1]
print(to_del)
# drop useless columns 
df.drop(to_del, axis=1, inplace=True) 
print(df.shape)

### Pertimbangkan Kolom yang Memeiliki Nilia Sangat Sedikit

Di bagian sebelumnya, kita melihat bahwa beberapa kolom dalam contoh dataset memiliki nilai unik yang sangat sedikit. 

Misalnya, ada kolom yang hanya memiliki 2, 4, dan 9 nilai unik. Ini mungkin masuk akal untuk variabel ordinal atau kategoris. Dalam hal ini, bagaimanapun, dataset hanya berisi variabel numerik. 

Dengan demikian, hanya memiliki 2, 4, atau 9 nilai numerik unik dalam sebuah kolom mungkin mengejutkan. 

Kita dapat menyebut kolom atau prediktor ini sebagai prediktor **varian mendekati nol**, karena variansnya bukan nol, tetapi *angka yang sangat kecil mendekati nol*.

Tergantung pada pilihan persiapan data dan algoritma pemodelan, variabel dengan nilai numerik yang sangat sedikit juga dapat menyebabkan kesalahan atau hasil yang tidak diharapkan. 

Sebagai contoh, saya telah melihat mereka menyebabkan kesalahan saat menggunakan transformasi daya untuk persiapan data dan ketika memasang model linier yang mengasumsikan distribusi probabilitas data yang masuk akal.

In [ ]:
# summarize the percentage of unique values for each column using numpy
from numpy import loadtxt
from numpy import unique
# load the dataset
data = loadtxt('https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv', delimiter=',')
# summarize the number of unique values in each column 
# data.shape --> (937,50)
# print(data.shape[0]) # get baris 
# print(data.shape[1]) # get kolom 
for i in range(data.shape[1]): # get kolom 
    num = len(unique(data[:, i]))
    percentage = float(num) / data.shape[0] * 100 
    if percentage < 1:
        print('Kurang dari 1 : %d, %d, %.1f%%' % (i, num, percentage))
    else:
        print('%d, %d, %.1f%%' % (i, num, percentage)) 

In [ ]:
# delete columns where number of unique values is less than 1% of the rows
from pandas import read_csv
# load the dataset
df = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv', header=None)
print(df.shape)

In [ ]:
# get number of unique values for each column
counts = df.nunique()
counts

In [ ]:
for i,v in enumerate(counts):
    if (float(v)/df.shape[0]*100) < 1:
        print(' : %d, %d, %.1f%%' % (i, v, float(v)/df.shape[0]*100))

In [ ]:

# record columns to delete
to_del = [i for i,v in enumerate(counts) if (float(v)/df.shape[0]*100) < 1] 
print(to_del)

In [ ]:
df.shape
#drop useless columns
#df.drop(to_del, axis=1, inplace=True)
#print(df.shape)

### Identify ROws That Conatain DUplicate Data

Di sini, baris duplikat adalah baris di mana setiap nilai di setiap kolom untuk baris itu muncul dalam urutan yang sama (nilai kolom yang sama) di baris lain.

... jika Anda telah menggunakan data mentah yang mungkin memiliki entri duplikat, menghapus data duplikat akan menjadi langkah penting untuk memastikan data Anda dapat digunakan secara akurat.

In [ ]:
# locate rows of duplicate data
from pandas import read_csv
# load the dataset
df = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv', header=None) # calculate duplicates
dups = df.duplicated()
dups

In [ ]:
# report if there are any duplicates 
print(dups.any())

In [ ]:
# list all duplicate rows 
print(df[dups])
print(df.shape)

In [ ]:
# delete duplicate rows 
#df.drop_duplicates(inplace=True) 
print(df.shape)

### How to Mark and Removing Missing Value 

Data dunia nyata sering kali memiliki nilai yang hilang. Data dapat memiliki nilai yang hilang karena sejumlah alasan seperti pengamatan yang tidak dicatat dan kerusakan data. 

Menangani data yang hilang itu penting karena banyak algoritme pembelajaran mesin tidak mendukung data dengan nilai yang hilang. Dalam hal ini kita akan menemukan cara menangani data yang hilang untuk pembelajaran mesin dengan Python.

* Cara menandai nilai yang tidak valid atau rusak sebagai hilang dalam kumpulan data Anda.
* Bagaimana mengkonfirmasi bahwa kehadiran nilai hilang yang ditandai menyebabkan masalah untuk algoritma Machine Learning.
* Cara menghapus baris dengan data yang hilang dari kumpulan data Anda dan mengevaluasi algoritme pembelajaran pada kumpulan data yang diubah.

### Diabetes Dataset

Dataset mengklasifikasikan data pasien baik sebagai onset diabetes dalam waktu lima tahun atau tidak. Ada 768 contoh dan delapan variabel input.

In [ ]:
# load and summarize the dataset
from pandas import read_csv
# load the dataset
dataset = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv', header=None) # summarize the dataset
dataset.head()
#dataset.shape

### Tandai Nilai yang Hilang

Sebagian besar data memiliki nilai yang hilang, dan kemungkinan nilai yang hilang meningkat seiring dengan ukuran kumpulan data.
Data yang hilang tidak jarang dalam kumpulan data nyata. Faktanya, peluang bahwa setidaknya satu titik data hilang meningkat seiring dengan bertambahnya ukuran kumpulan data.


Di bagian ini, kita akan melihat bagaimana kita dapat mengidentifikasi dan menandai nilai sebagai hilang. Kita dapat menggunakan plot dan ringkasan statistik untuk membantu mengidentifikasi data yang hilang atau rusak.

In [ ]:
print(dataset.describe())

Kita dapat melihat bahwa ada kolom yang memiliki nilai minimal nol (0). Pada beberapa kolom, nilai nol tidak masuk akal dan menunjukkan nilai yang tidak valid atau hilang.
Nilai yang hilang sering ditunjukkan oleh entri di luar rentang; mungkin angka negatif (misalnya, -1) di bidang numerik yang biasanya hanya positif, atau 0 di bidang numerik yang biasanya tidak pernah menjadi 0.

Specifically, the following columns have an invalid zero minimum value:
* 1: Plasma glucose concentration 
* 2: Diastolic blood pressure
* 3: Triceps skinfold thickness
* 4: 2-Hour serum insulin
* 5: Body mass index

In [ ]:
print(dataset.head(20))
#kita lihat 0 values pada columns 2,3,4 and 5

Kita bisa mendapatkan hitungan jumlah nilai yang hilang pada masing-masing kolom ini. Kita dapat melakukan ini dengan menandai semua nilai dalam subset DataFrame yang kita minati yang memiliki nilai nol sebagai True. kita kemudian dapat menghitung jumlah nilai sebenarnya di setiap kolom.

In [ ]:
num_missing = (dataset[[1,2,3,4,5]] == 0).sum()
# report the results
print(num_missing)

In [ ]:
from numpy import nan
# replace '0' values with 'nan'
dataset[[1,2,3,4,5]] = dataset[[1,2,3,4,5]].replace(0, nan) # count the number of nan values in each column print(dataset.isnull().sum())

In [ ]:
# summarize the first 20 rows of data 
print(dataset.head(20))

### Remove Rows With Missing Values

In [ ]:
print(dataset.shape)

In [ ]:
# drop rows with missing values
dataset.dropna(inplace=True)

In [ ]:
# summarize the shape of the data with missing rows removed 
print(dataset.shape)

In [ ]:
# example where missing values cause errors
from numpy import nan
from pandas import read_csv
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# load the dataset
dataset = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv', header=None) # summarize the dataset
# replace '0' values with 'nan'
dataset[[1,2,3,4,5]] = dataset[[1,2,3,4,5]].replace(0, nan)
# split dataset into inputs and outputs
values = dataset.values
X = values[:,0:8]
y = values[:,8]
# define the model
model = LinearDiscriminantAnalysis()
# # define the model evaluation procedure
cv = KFold(n_splits=3, shuffle=True, random_state=1)
# # evaluate the model
result = cross_val_score(model, X, y, cv=cv, scoring='accuracy') 
# # report the mean performance
print('Accuracy: %.3f' % result.mean())

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
        
Error caused by the presence of missing values.

Ini seperti yang kita harapkan. Kami dicegah untuk mengevaluasi algoritme LDA (dan algoritme lainnya) pada kumpulan data dengan nilai yang hilang.
Banyak model prediktif populer seperti vector machines, the glmnet, and neural networks, tidak dapat mentolerir sejumlah nilai yang hilang.

Remove Rows With Missing Values


Strategi paling sederhana untuk menangani data yang hilang adalah dengan menghapus catatan yang berisi nilai yang hilang.
Pendekatan paling sederhana untuk menangani nilai yang hilang adalah dengan menghapus seluruh prediktor dan/atau sampel yang mengandung nilai yang hilang.

Kita dapat melakukan ini dengan membuat DataFrame Pandas baru dengan menghapus baris yang berisi nilai yang hilang. 

Pandas menyediakan fungsi dropna() yang dapat digunakan untuk menjatuhkan kolom atau baris dengan data yang hilang. Kita dapat menggunakan dropna() untuk menghapus semua baris dengan data yang hilang, sebagai berikut:

In [ ]:
# example where missing values cause errors
from numpy import nan
from pandas import read_csv
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# load the dataset
dataset = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv', header=None) # summarize the dataset
# replace '0' values with 'nan'
dataset[[1,2,3,4,5]] = dataset[[1,2,3,4,5]].replace(0, nan)
# drop rows with missing values
dataset.dropna(inplace=True)
# split dataset into inputs and outputs
values = dataset.values
X = values[:,0:8]
y = values[:,8]
# define the model
model = LinearDiscriminantAnalysis()
# # define the model evaluation procedure
cv = KFold(n_splits=3, shuffle=True, random_state=1)
# # evaluate the model
result = cross_val_score(model, X, y, cv=cv, scoring='accuracy') 
# # report the mean performance
print('Accuracy: %.3f' % result.mean())

### Analyzing Obesity in England

In [ ]:
pip install matplotlib

In [ ]:
import matplotlib as mpl

In [ ]:
pip install --upgrade pip

In [ ]:
pip install xlrd>=1.0.0

In [ ]:
%matplotlib inline
data = pd.ExcelFile('obes.xls')


In [ ]:
data.sheet_names

Kita akan fokus pada sheet 7.2, sekarang jika melihat table 7.2 pada excel, kita akan melihat bahwa 4 baris teratas dan 14 baris terbawah berisi info yang tidak berguna, kita hanya membutuhkan baris 5-18

In [ ]:
data_age = data.parse(u'7.2', skiprows=4, skipfooter=14)
data_age.head()

Baris pertama mewakili columns headers. Kita memiliki baris kosong dile asli, dan itu, muncul sebagai NaN (Bukan angka)

Jadi sekrang kita perlu melakukan 2 hal:
- Rename the first headr to Year, and
- Get rid of any empty rows

In [ ]:
# mengganti nama kolom Unnamed: 0 menjadi year
data_age.rename(columns={u'Unnamed: 0':u'Year'}, inplace=True)


In [ ]:
# menghapus baris kosong yang diisi dengan NaN
data_age.dropna(inplace=True)


In [ ]:
data_age.head()

Jika kita melihat data_age, nilai pertama adalah angka. Ini adalah indeksnya, dan pandas menggunakan paraktik excel default yang memiliki angka sebagia indeks. Kita akan mengubah indeks menjadi Tahun. ini akan mempermudak pembuatan plot, karena indeks biasanya diplot sebagai sumbu x

In [ ]:
data_age.set_index('Year', inplace = True)

In [ ]:
data_age.plot()

Di lihat dari grafik di atas, jika di lihat ada yang salah yaitu data asli kita berisi total yang menutupi bidang lainnya. Kita perlu mentingkitkannya

In [ ]:
# Hapus kolom Total dari datase
data_age_minus_total = data_age.drop('Total', axis = 1)

In [ ]:
# plot ulang
data_age_minus_total.plot()

Jauh lebih baik dari sbelumnya, kita benar-benar dapat melihat kelompok individu sekrang. Bisakah kita melihat kelompok usia mana yang memiliki obesitas tertinggi?

Kembali ke peryantaan awal kita: Are children getting fatter?

Mari plot sebagian kecil data: anak-anak dibawah usia 16 tahun dan orang dewasa dengan usia 35-44 tahun

In [ ]:
data_age['Under 16'].plot(label = 'Under 16', legend = True)
data_age['35-44'].plot(label = '35-44', legend=True)

Dari grafik diatas dapat disimpulkan, ketika obesitas anak sedikit mennurun, orang tua terus membengkak. Jadi nampkanya para orang tua harus menghawatirkan diri sendiri daripada menghawatirkan anaknya

### Time Series
### Basic Time Series Manipulation


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from datetime import datetime

In [ ]:
data_rng = pd.date_range(start='1/01/2020', end='1/08/2020', freq='H')

In [ ]:
data_rng

Mari kita buat contoh data frame dengan timestamp dan lihat 15 elemen pertama

In [ ]:
df = pd.DataFrame(data_rng, columns=['date'])

In [ ]:
df['data'] = np.random.randint(0,100,size=(len(data_rng)))

In [ ]:
df

Jika kita ingin melakukan manipulasi deret waktu/time series, kita perlu memiliki date time index sehingga data frame kita di indeks pada timestamp

Konversikan indeks data frame menjadi datetime index dan tampilkan elemen pertama

In [ ]:
df['datetime'] = pd.to_datetime(df['date'])
df = df.set_index('datetime')
df.drop(['date'], axis = 1, inplace=True)
df.head()

Bagaimana jika 'time' stamps dalam data kita sebernarnya berjenid string vs numberik?

Mari kita ubah data_rng kita menjadi list of string dan kemudian ubah string tersebut menjadi stempel waktu

In [ ]:
string_data_rng = [str(x) for x in data_rng]
string_data_rng

Kita dapat mengkonversi string menjadi timestamps dengan melihat formatnya, lalu melihat nilainya

In [ ]:
timestamp_data_rng = pd.to_datetime(string_data_rng,
                                    infer_datetime_format=True)

In [ ]:
timestamp_data_rng

Tetapi bagaimana jika perlu mengubah format unique string

Mari kita buat daftar tunggal yang berubah-ibah menjadi string dan mengubahnya menjadi timestamps

In [ ]:
string_data_rng_2 = ['June-01-2020', 'June-02-2020', 'June-03-2020']

In [ ]:
timestamp_data_rng_2 = [datetime.strptime(x, '%B-%d-%Y') for x in string_data_rng_2]

In [ ]:
timestamp_data_rng_2

seperti apa jadinya jika kita memasukan ini ke dalam data frame

In [ ]:
df2 = pd.DataFrame(timestamp_data_rng_2, columns=['date'])
df2

kembali ke data frame asli kita, mari kita lihat datanya dengan parsing pada timestamp index:

Katakanlah kita hanya ingin melihat data di mana tanggalnya adalah tanggal 2 setiap bulannya, kita bisa menggunakan indeks seperti dibawah ini

In [ ]:
df[df.index.day == 2]

Kita juga bisa langsung memanggil tanggal yang ingin kita lihat memaui index dari data frame

In [ ]:
df['2020-01-03']

Bagaimana dengan memilih data di antara tanggal tertentu

In [ ]:
df['2020-01-04': '2020-01-06']

Basic data frame yang telah kita buat memberi kita data dengan frekuensi per jam, tetapi kita dapat mengambil ulang sampel/resample data pada frekuensi yang berbeda dan menentukan bagaimana menentukan bagaimana kita ingin menghitung summary statistic untuk frekuensi sample baru

kita dapat mengambil min, max, average,sum, dll, dari data frekuensi harian dari pada frekuensi per jam seperti contoh di bawah ini tempat kita menghitung rata-rata harian dari data:


In [ ]:
df.resample('D').mean()

Bagaimana dengan window statistics seperti rolling mean atau rolling sum?

Mari buat kolom baru di df asli kota yang menghitung rolling sum selama periode 3 window dan kemudian lihat di bagina atas data frame


In [ ]:
df['rolling_sum'] = df.rolling(3).sum()

In [ ]:
df.head()

Kita dapat melihat bahwa pandas menghitung dengan benar dan hanya memiliki nilai yang valid ketika ada tiga periode untuk melihat ke belakang

Ini adalah kesempatan bagus untuk melihat bagaimana kita dapat melakukan forwarding of backfilling data saar bekerja dengan nilai data yang hilang

Brikut df kita, tetapi dengan kolom bari yang mengabil rolling sum dan backfills data

In [ ]:
df['rolling_sum_backfilled'] = df['rolling_sum'].fillna(method = 'backfill')

In [ ]:
df.head()

### Time Series on Dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
opsd_daily = pd.read_csv('https://raw.githubusercontent.com/ardhiraka/PFDS_sources/master/opsd_germany_daily.csv')
opsd_daily.shape

In [ ]:
opsd_daily.head()

In [ ]:
opsd_daily.tail()

kolom Date adalah tipe data yang benar, mari kita set sebagai indeks dataframe

In [ ]:
opsd_daily = opsd_daily.set_index('Date')
opsd_daily.head(3)

In [ ]:
opsd_daily.index

alternatifnya, kita dapat menggabungkan langkah-langkah di atas menjadi satu baris, menggunakan parameter index_col dan parse_dates dari pungsi read.csv().

In [ ]:
opsd_daily = pd.read_csv('https://raw.githubusercontent.com/ardhiraka/PFDS_sources/master/opsd_germany_daily.csv', index_col=0, parse_dates=True)

In [ ]:
opsd_daily.head()

Aspek berguna lainnya dari DatetimeIndex adalah bahwa setiap komponen tanggal / waktu tersedia sebagai atribut seperti tahun, bulan, hari dan setrusnya.Mari tambahkan beberapa kolom lagi ke opsd_daily yang berisi tahun, bulan dan hari kerja

In [ ]:
opsd_daily['Year'] = opsd_daily.index.year
opsd_daily['Month'] = opsd_daily.index.month
opsd_daily['Weekday'] = opsd_daily.index.weekday

In [ ]:
opsd_daily.head()

Kita dapat memilih data untuk satu hari menggunakan string seperti '2017-08-10

In [ ]:
opsd_daily.loc['2017-08-10']

Kita juga dapat memilih slice hari, seperti '2014-01-20':'2014-01-22'. Seperti pengindeksan berbasis label biasa dengan loc, slice tersebut menyertakan kedua titik akhir

In [ ]:
opsd_daily.loc['2014-01-20':'2014-01-22']

Fitur lain yang sangat berguna dari time series pandas adalah partial-string indexing, dimana kita dapat memilih semua tanggal / waktu yang sebagian cocok denagan string yang di berikan , misalnya kita dapat memilih seluruh tahun 2006 dengan opsd_daily.loc['2006']
, atau seluruh selurh bulan februari 2012 dengan opsd_daily.loc['2012-02']

In [ ]:
opsd_daily.loc['2012-02']

Mari kita buat ine plot dari time series komsumsi listrik harian di jerman, menggunakan plot() dataframe

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
opsd_daily['Consumption'].plot(linewidth=0.5)

Kita dapat melihat bahwa method plot() memilih lokasi tick yang cukup bagus(setiap dua tahun) dan label(years) untuksumbu x, yang sangat membantu. Namun, dengan banyaknya tiitk data, line plot menjadi padat dan sulit untuk dibaca 

Mari kita plot data sebagian titik-titik dan lihat juga time series dan matahari dan angin

In [ ]:
opsd_daily['Consumption'].plot(marker='.', alpha=0.5, linestyle='None',
                              figsize=(11,9));

In [ ]:
opsd_daily['Solar'].plot(marker='.', alpha=0.5, linestyle='None', figsize=(11,9))

In [ ]:
opsd_daily['Wind'].plot(marker='.', alpha=0.5, linestyle='None', figsize=(11,9))

Kita sudah bisa melihat beberapa pola menarik yang muncul:

- Konsumsi listrik tertinggi ada di musim dingin, kemungkinan karena pemanas listrik dan peningkatan penggunaan penerangan, dan terendah di musim panas.

- Produksi tenaga surya paling tinggi di musim panas, saat sinar matahari paling melimpah, dan paling rendah di musim dingin.

- Produksi tenaga angin paling tinggi di musim dingin, mungkin karena angin yang lebih kuat dan badai yang lebih sering, dan paling rendah di musim panas.

- Tampaknya ada tren peningkatan yang kuat dalam produksi tenaga angin selama bertahun-tahun.

Ketiga time series dengan jelas menunjukkan periodisitas — sering disebut sebagai seasonality dalam time series analysis — di mana sebuah pola berulang berulang kali pada interval waktu yang teratur. 

Rangkaian waktu Consumption, Solar, dan Wind berosilasi antara nilai tinggi dan rendah dalam skala waktu tahunan, sesuai dengan perubahan cuaca musiman sepanjang tahun. 

Namun, musim secara umum tidak harus sesuai dengan musim meteorologi. Misalnya, data penjualan ritel sering kali menunjukkan musim tahunan dengan peningkatan penjualan pada bulan November dan Desember, menjelang liburan.
Seasonality juga dapat terjadi pada skala waktu lain. Plot di atas menunjukkan bahwa konsumsi listrik Jerman mungkin terlihat weekly seasonality, sesuai dengan hari kerja dan akhir pekan. 

Mari kita plot deret waktu dalam satu tahun untuk menyelidiki lebih lanjut.

In [ ]:
ax = opsd_daily.loc['2017', 'Consumption'].plot(figsize=(11,9))
ax.set_ylabel('Daily Consumption(GWH)')

Sekarang kita dapat dengan jelas melihat osilasi mingguan/weekly oscillations. Ciri menarik lain yang terlihat pada tingkat granularitas ini adalah penurunan drastis konsumsi listrik pada awal Januari dan akhir Desember, selama liburan.

Mari kita perbesar lebih jauh dan lihat bulan Januari dan Februari saja.

In [ ]:
ax = opsd_daily.loc['2017-01':'2017-02', 'Consumption'].plot(marker='o',
                                                             linestyle='-', figsize=(11,9))
ax.set_ylabel('Daily COnsumption (GWH)')


Selanjutnya, mari kita jelajahi lebih jauh seasonality data kita dengan fungsi box plots untuk mengelompokkan data berdasarkan periode waktu yang berbeda dan menampilkan distribusi untuk setiap kelompok. Pertama-tama, kita akan mengelompokkan data berdasarkan bulan, untuk memvisualisasikan yearly seasonality.

In [ ]:
opsd_daily.boxplot(column=['Consumption'], by='Month')

In [ ]:
opsd_daily.boxplot(column=['Wind'], by='Month')

In [ ]:
opsd_daily.boxplot(column=['Solar'], by='Month')

box plots berikut mengonfirmasi yearly seasonality yang kitalihat di plot sebelumnya dan memberikan beberapa insights:

Meskipun konsumsi listrik umumnya lebih tinggi di musim dingin dan lebih rendah di musim panas, 

median dan dua kuartil lebih rendah pada bulan Desember dan Januari dibandingkan dengan November dan Februari, kemungkinan karena bisnis tutup selama liburan. 

Kita melihat ini dalam rangkaian waktu untuk tahun 2017, dan box plot menegaskan bahwa ini adalah pola yang konsisten selama bertahun-tahun.

Sementara produksi tenaga surya dan angin sama-sama menunjukkan yearly seasonality, distribusi tenaga angin memiliki lebih banyak outliers, yang mencerminkan efek kecepatan angin ekstrem sesekali yang terkait dengan badai dan kondisi cuaca sementara lainnya.

Selanjutnya, mari kelompokkan rangkaian waktu konsumsi listrik berdasarkan hari dalam seminggu, untuk menjelajahi weekly seasonality.

In [ ]:
opsd_daily.boxplot(column=['Consumption'], by='Weekday')

Seperti yang diharapkan, konsumsi listrik secara signifikan lebih tinggi pada hari kerja dibandingkan pada akhir pekan. Outliers rendah pada hari kerja mungkin selama hari libur/holidays.

Seringkali berguna untuk resample data time series kita ke frekuensi yang lebih rendah atau lebih tinggi. 

Resampling ke frekuensi yang lebih rendah (downsampling) biasanya melibatkan operasi agregasi - misalnya, menghitung total penjualan bulanan dari data harian. 

Resampling ke frekuensi yang lebih tinggi (upsampling) kurang umum dan sering kali melibatkan interpolasi atau metode pengisian data lainnya - misalnya, menginterpolasi data cuaca setiap jam hingga interval 10 menit untuk dimasukkan ke model ilmiah.

Kita akan fokus di sini pada downsampling, mengeksplorasi bagaimana hal itu dapat membantu kita menganalisis data OPSD dalam berbagai skala waktu. 

Kita menggunakan method resample() DataFrame, yang membagi DatetimeIndex ke dalam time bins dan mengelompokkan data menurut time bin. 

Metode resample() mengembalikan objek Resampler, mirip dengan objek GroupBy pandas. 

Kita kemudian dapat menerapkan metode agregasi seperti  mean(), median(), sum(), dll., Ke grup data untuk setiap time bin.

Misalnya, mari kita resample data menjadi weekly mean time series.

In [ ]:
data_columns = ['Consumption','Wind','Solar','Wind+Solar']
opsd_weekly_mean = opsd_daily[data_columns].resample('W').mean()
opsd_weekly_mean.head(3)

Baris pertama di atas, berlabel 2006-01-01, berisi rata-rata dari semua data yang ada dalam time bin 2006-01-01 hingga 2006-01-07. Baris kedua, berlabel 2006-01-08, berisi data rata-rata untuk time bin 2006-01-08 hingga 2006-01-14, dan seterusnya.

Berdasarkan konstruksi, deret waktu mingguan kita memiliki 1/7 poin data sebanyak deret waktu harian. Kita dapat mengonfirmasi ini dengan membandingkan jumlah baris dari dua DataFrame.

In [ ]:
print(opsd_daily.shape[0])
print(opsd_weekly_mean.shape[0])

Mari kita ganbarkan rangkaianwaktu solar harian dan mingguan dalam satu periode enam bulan untuk membandingkannya

In [ ]:
start, end = '2017-01', '2017-06'

opsd_daily.loc[start:end, 'Solar'].plot(marker='.', linestyle='-',
                                        linewidth=0.5, figsize=(11,9))
opsd_weekly_mean.loc[start:end, 'Solar'].plot(marker='o', markersize=8,
                                              linestyle='-', figsize=(11,9))

Kita dapat melihat bahwa deret waktu rata-rata mingguan lebih halus daripada deret waktu harian karena variabilitas frekuensi yang lebih tinggi telah dirata-ratakan dalam resampling.

In [ ]:
start, end = '2017-01', '2017-06'

opsd_daily.loc[start:end, 'Wind'].plot(marker='.', linestyle='-',
                                        linewidth=0.5, figsize=(11,9))
opsd_weekly_mean.loc[start:end, 'Wind'].plot(marker='o', markersize=8,
                                              linestyle='-', figsize=(11,9))